In [1]:
import ml
reload(ml)
from ml import *
import timeit
import scipy
import operator
import collections
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from collections import Counter
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import statsmodels.api as sm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.ensemble import AdaBoostClassifier as ADB
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.tree import DecisionTreeClassifier as CART
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.manifold.t_sne import TSNE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sns.set_style('whitegrid')
plt.style.use('seaborn-whitegrid')

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

In [2]:
data = pd.read_csv("../Data/BidLevelData.txt", sep="|")
data.sample(5)

,ListingKey,HowManythBid,AmountRequested,AvgInterBidTime,FundedOrNot,RepaidOrNot,Gini,NoBids,BorrowerState,Year,CreditGrade,DebtToIncomeRatio,DescriptionLength
235762,0649336712422710175AF8F,256,9000.000,271.219,True,True,0.404,354,GA,2006,C,0.270,73
4986924,82CE351074455404235446F,27,9000.000,15106.407,True,NaN,0.270,226,MI,2011,NaN,0.140,88
1305320,221535250030198642E0EEC,72,5500.000,16308.444,False,NaN,0.319,103,FL,2011,NaN,nan,69
1250267,20A1341629243942243690C,163,9924.000,589.025,True,True,0.478,322,GA,2008,B,0.260,332
7552596,C7213390428121060ABCA17,3,12500.000,345.667,True,True,0.019,108,AZ,2007,B,0.300,237


In [3]:
f = {'ListingKey': ['max'],
     'NoBids':['max'], 
     'AmountRequested':['max'], 
     'AvgInterBidTime': ['median'],
     'Gini': ['median'],
     'DebtToIncomeRatio': ['max'],
     'DescriptionLength': ['max'],
     'RepaidOrNot': ['max']}

all_features = ["ListingKey","NoBids", "AmountRequested", "AvgInterBidTime", "Gini", 
                "DebtToIncomeRatio", "DescriptionLength", "RepaidOrNot"]
train_features = ["NoBids", "AmountRequested", "AvgInterBidTime", "Gini", 
                  "DebtToIncomeRatio", "DescriptionLength"]

## Funded Data

In [4]:
funded_data = pd.DataFrame(data.loc[(data['RepaidOrNot']==True) | (data['RepaidOrNot']==False)][all_features].groupby('ListingKey').agg(f).as_matrix())
funded_data = funded_data.rename(index=str, columns={0: "AmountRequested", 
                                                       1: "Gini", 
                                                       2: "DebtToIncomeRatio",
                                                       3: "AvgInterBidTime",
                                                       4: "NoBids",
                                                       5: "DescriptionLength",
                                                       6: "RepaidOrNot"})
funded_data = funded_data.drop(7, axis=1)
funded_data.sample(5)

,AmountRequested,Gini,DebtToIncomeRatio,AvgInterBidTime,NoBids,DescriptionLength,RepaidOrNot
32438,9000.000,0.451,0.130,4408.781,88,153,True
22770,1500.000,0.259,NaN,11210.244,287,86,True
21577,2500.000,0.090,0.130,325538.125,90,8,False
19363,3000.000,0.093,0.330,22.205,171,69,True
15733,3000.000,0.494,0.180,19124.802,327,34,True


In [5]:
print ("Percentage not repaid from funded listings: %")
print (dict(Counter(funded_data.RepaidOrNot))[False] / len(funded_data)) * 100 

Percentage not repaid from funded listings: %
34.5345345345


## Unfunded Data

In [6]:
unfunded_data = pd.DataFrame(data.loc[data['FundedOrNot']==False][all_features].groupby('ListingKey').agg(f).as_matrix())
unfunded_data = unfunded_data.rename(index=str, columns={0: "AmountRequested", 
                                                       1: "Gini", 
                                                       2: "DebtToIncomeRatio",
                                                       3: "AvgInterBidTime",
                                                       4: "NoBids",
                                                       5: "DescriptionLength"})
unfunded_data = unfunded_data.drop([6,7], axis=1)
unfunded_data.sample(5)

,AmountRequested,Gini,DebtToIncomeRatio,AvgInterBidTime,NoBids,DescriptionLength
86600,3000.000,0.083,0.290,424333.750,166,2
167798,4900.000,0.326,0.210,31670.028,188,44
60873,9000.000,0.000,0.390,603591.000,207,1
12527,1600.000,0.000,NaN,864121.000,132,1
55023,25000.000,0.000,0.320,520.000,313,1


## Test Set

In [7]:
X_test = unfunded_data[train_features].fillna(-1).as_matrix()

In [8]:
RF = RF(random_state=42) # Random Forest Classifier
scaler = StandardScaler() # Standard Scaler

## Imbalanced Classes

In [9]:
X_train = funded_data[train_features].fillna(-1).as_matrix()
X_train = scaler.fit_transform(X_train)

y_train = funded_data["RepaidOrNot"].astype(int)

RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [10]:
y_pred = RF.predict(X_test)

In [11]:
print ("Percentage not repaid from unfunded listings: %")
print (dict(Counter(y_pred))[0] / len(y_pred)) * 100 

Percentage not repaid from unfunded listings: %
52.6076161189


## Validation

In [12]:
y_pred = run_cv(X_train, y_train, RF)

print "Accuracy: ", accuracy(y_train, y_pred)
print "Precision: ", precision(y_train, y_pred)
print "Recall: ", recall(y_train, y_pred)
print "F-Measure: ", f1(y_train, y_pred)
print "AUC/ROC: ",  auc(y_train, y_pred)

Accuracy:  0.6092563151386681
Precision:  0.6905936383738731
Recall:  0.7303471847454578
F-Measure:  0.7099143206854345
AUC/ROC:  0.555028664836497


## Feature Ranking

In [13]:
frank_summary(X_train, y_train, train_features)

,Rank,Feature,Importance,Support,x
0,1,NoBids,0.177,True,1
1,1,AmountRequested,0.137,True,2
2,1,DescriptionLength,0.178,True,6
3,2,DebtToIncomeRatio,0.163,False,5
4,3,AvgInterBidTime,0.174,False,3
5,4,Gini,0.170,False,4


## Balanced Classes

In [14]:
from sklearn.utils import resample

# Separate majority and minority classes
funded_majority = funded_data[funded_data.RepaidOrNot==True]
funded_minority = funded_data[funded_data.RepaidOrNot==False]

funded_majority_downsampled = resample(funded_majority, 
                                         replace=False,    # sample without replacement
                                         n_samples=11730,  # to match minority class
                                         random_state=123) # reproducible results

# Combine minority class with downsampled majority class
funded_downsampled = pd.concat([funded_majority_downsampled, funded_minority])
 
# Display new class counts
funded_downsampled.RepaidOrNot.value_counts()

True     11730
False    11730
Name: RepaidOrNot, dtype: int64

In [15]:
funded_downsampled.sample(5)

,AmountRequested,Gini,DebtToIncomeRatio,AvgInterBidTime,NoBids,DescriptionLength,RepaidOrNot
30944,3000.000,0.389,0.350,16390.922,299,102,True
17062,3200.000,0.349,0.260,14646.060,365,46,True
33538,3700.000,0.377,0.130,4769.868,365,151,True
8464,9800.000,0.588,0.260,2148.144,271,391,True
6260,5000.000,0.389,0.480,7560.851,321,182,True


In [16]:
X_train = funded_downsampled[train_features].fillna(-1).as_matrix()
X_train = scaler.fit_transform(X_train)

y_train = funded_downsampled["RepaidOrNot"].astype(int)

RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [17]:
y_pred = RF.predict(X_test)

In [18]:
print ("Percentage not repaid from unfunded listings: %")
print (dict(Counter(y_pred))[0] / len(y_pred)) * 100 

Percentage not repaid from unfunded listings: %
73.2088429128


## Listing Summary

In [19]:
print "Funded Listings: %", len(funded_data) / len(data.groupby('ListingKey')) * 100
print "Repaid Loans: %", len(funded_data[funded_data.RepaidOrNot == True]) / len(funded_data) * 100
print "Defaulted Loans: %", len(funded_data[funded_data.RepaidOrNot == False]) / len(funded_data) * 100
print "Failed Listings: %", len(unfunded_data) /len(data.groupby('ListingKey')) * 100

Funded Listings: % 14.407451867
Repaid Loans: % 65.4654654655
Defaulted Loans: % 34.5345345345
Failed Listings: % 78.07196515


In [20]:
funded_data.describe()

,AmountRequested,Gini,DebtToIncomeRatio,AvgInterBidTime,NoBids,DescriptionLength,RepaidOrNot
count,33966.000,33966.000,32111.000,33966.000,33966,33966,33966
unique,1030.000,32552.000,1124.000,33860.000,731,844,2
top,5000.000,0.000,0.180,0.000,57,1,True
freq,3183.000,477.000,1095.000,22.000,248,338,22236


In [21]:
unfunded_data.describe()

,AmountRequested,Gini,DebtToIncomeRatio,AvgInterBidTime,NoBids,DescriptionLength
count,184057.000,184057.000,165841.000,184057.000,184057,184057
unique,2202.000,55009.000,3694.000,159252.000,767,747
top,5000.000,0.000,0.200,3.000,57,1
freq,20211.000,90573.000,4653.000,553.000,2646,64848
